In [434]:
!pwd

/Users/jryzu/mle_in_100_days/day_2/ppo_from_scratch


python(95768) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [1]:
# 2. Implement PPO

In [56]:
# !pip install gymnasium

In [57]:
# !pip install swig "gymnasium[box2d]"

In [4]:
import gymnasium as gym

In [5]:
env = gym.make('Pendulum-v1', render_mode='human')

In [6]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [7]:
env.action_space.shape

(1,)

In [8]:
env.observation_space.shape

(3,)

In [12]:
env.observation_space

Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)

In [17]:
env.action_space.sample()

array([1.2128432], dtype=float32)

In [55]:
# for i in range(1000):
#     print(env.step([0]))

In [42]:
# # Reset the environment to generate the first observation
# observation, info = env.reset(seed=42)
# for _ in range(1000):
#     # this is where you would insert your policy
#     action = env.action_space.sample()

#     # step (transition) through the environment with the action
#     # receiving the next observation, reward and if the episode has terminateda or truncated
#     observation, reward, terminated, truncated, info = env.step(action)

#     # If the episode has ended then we can reset to start a new episode
#     if terminated or truncated:
#         observation, info = env.reset()

# env.close()

In [76]:
env.reset()

(array([-0.00236187,  1.413665  , -0.23925176,  0.12199411,  0.00274366,
         0.05419413,  0.        ,  0.        ], dtype=float32),
 {})

In [77]:
env.render()

In [81]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [58]:
env.observation_space

Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)

In [89]:
timesteps_per_batch = 1000
max_ep_len = 1000

In [90]:
# simulate one episode up to max length

In [91]:
observation, info = env.reset(seed=42)

In [92]:
observation

array([-0.14995256,  0.9886932 , -0.12224312], dtype=float32)

In [94]:
info

{}

In [95]:
action = env.action_space.sample()

In [97]:
action

array([-0.9419495], dtype=float32)

In [96]:
observation, reward, terminated, truncated, info = env.step(action)

In [100]:
terminated

False

In [101]:
truncated

False

In [103]:
info

{}

In [ ]:
# observation, info = env.reset(seed=42)
# for _ in range(1000):
#     # this is where you would insert your policy
#     action = env.action_space.sample()

#     # step (transition) through the environment with the action
#     # receiving the next observation, reward and if the episode has terminateda or truncated
#     observation, reward, terminated, truncated, info = env.step(action)

#     # If the episode has ended then we can reset to start a new episode
#     if terminated or truncated:
#         observation, info = env.reset()

# env.close()

In [108]:
env.reset()[0].shape

(3,)

In [195]:
import datetime
from torch.optim import Adam

In [250]:
env.action_space.shape

(1,)

In [252]:
torch.full(size=(env.action_space.shape[0],), fill_value=0.5)

tensor([0.5000])

In [254]:
torch.diag(torch.full(size=(env.action_space.shape[0],), fill_value=0.5))

tensor([[0.5000]])

In [422]:
class PPO: # asked ChatGPT to give me the class skeleton
    def __init__(self, policy_cls, env):
        self.obs_dim = env.observation_space.shape[0]
        self.act_dim = env.action_space.shape[0]
        self.actor = policy_cls(self.obs_dim, self.act_dim)
        self.critic = policy_cls(self.obs_dim, 1) # for the reward
        
        self.lr = 0.001
        
        self.actor_optim = Adam(self.actor.parameters(), lr=self.lr)
        self.critic_optim = Adam(self.critic.parameters(), lr=self.lr)
        
        self.cov_var = torch.full(size=(self.act_dim,), fill_value=0.4)
        self.cov_mat = torch.diag(self.cov_var)
        
        self.max_timesteps = 400
        self.n_updates_per_iter = 5
        self.clip = 0.1
        
        self.last_batch_rews = 0
        
    def get_action(self, obs):
        mean = self.actor.forward(torch.tensor(obs))
        dist = MultivariateNormal(mean, self.cov_mat)
        action = dist.sample().detach()
        return action, dist.log_prob(action).detach() # error caught by Claude - initially resampled
    
    def rollout(self, timesteps_per_batch=1048):
        batch_obs = []
        batch_acts = []
        batch_log_probs = []
        
        batch_lens = []
        batch_rews = []
        batch_rtgs = []
        collected = 0
        
        while collected < timesteps_per_batch:
            print(datetime.datetime.now())
            print(collected)
            ep_t = 0 
            ep_rews = []
            obs, _ = env.reset()
            terminated, truncated = False, False
            while not terminated and not truncated and ep_t < self.max_timesteps:
                act, log_prob = self.get_action(obs)
                batch_obs.append(obs)
                obs, rew, terminated, truncated, _ = env.step(act) # error caught by Claude - have to store obs before step
                batch_acts.append(act)
                batch_log_probs.append(log_prob)
#                 rtgs = # consulted 
#                 batch_lens = 
                ep_t += 1
                ep_rews.append(rew)
                collected += 1
            batch_lens.append(ep_t)
            batch_rews.append(ep_rews)
            
        batch_obs = torch.tensor(batch_obs, dtype=torch.float)
        batch_acts = torch.tensor(batch_acts, dtype=torch.float)
        batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float)
        batch_rtgs = self.compute_rtgs(batch_rews)
        
        self.last_batch_rews = batch_rews
        return batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens
            
    @staticmethod # learned something new here
    def compute_rtgs(batch_rews, gamma=0.9): # consulted ChatGPT
        batch_rtgs = []
        for ep_rews in reversed(batch_rews):
            discounted_reward = 0
            for _, rew in enumerate(reversed(ep_rews)):
                discounted_reward = gamma * discounted_reward + rew
                batch_rtgs.insert(0, discounted_reward)
                
        batch_rtgs = torch.tensor(batch_rtgs, dtype = torch.float)
        return batch_rtgs
    
    def evaluate(self, batch_obs, batch_acts):
        V = self.critic(batch_obs).squeeze()
        if batch_acts.dim() == 1:
            batch_acts = batch_acts.unsqueeze(-1) # needed Claude's help for this since learn() stuck on this
        log_probs = MultivariateNormal(self.actor(batch_obs), self.cov_mat).log_prob(batch_acts)
        return V, log_probs
        
    def learn(self, n_timesteps = 100000):
        t=0
        iteration=0
        while t<n_timesteps:
            obs, acts, old_logp, rtgs, lens = self.rollout()       
            t += sum(lens) # bug caught by Claude
            
            # Add this logging:
            avg_return = sum([sum(ep_rews) for ep_rews in self.last_batch_rews]) / len(lens)
            print(f"Iteration {iteration}: Timesteps {t}, Avg Return: {avg_return:.2f}, Episodes: {len(lens)}")
            
            A = (rtgs - self.evaluate(obs, acts)[0]).detach()
            # advantage normalization step - not yet implemented
            A = (A - A.mean()) / (A.std() + 1e-8)
            
            for i in range(self.n_updates_per_iter):
                print('iteration ', i)
                V, logp = self.evaluate(obs, acts)
                ratio = torch.exp(logp-old_logp)
                surr1 = ratio * A # why is A frozen here
                surr2 = torch.clamp(ratio, 1-self.clip, 1+self.clip) * A
                actor_loss = -torch.min(surr1, surr2).mean() # this is fine, recall PyTorch notes down a graph for backprop
                critic_loss = F.mse_loss(V, rtgs)
                
                self.actor_optim.zero_grad()
                actor_loss.backward(retain_graph=True)
                torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 0.5)  # Add this
                self.actor_optim.step()
                
                self.critic_optim.zero_grad()
                critic_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.critic.parameters(), 0.5)  # Add this
                self.critic_optim.step()
                
            iteration += 1
                
        # train the critic on A
        # train the actor on A * 

In [423]:
model = PPO(FeedForwardNN, env)

In [424]:
obs, _ = env.reset()

In [425]:
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.normal import Normal

In [426]:
model.get_action(obs)

(tensor([-0.2696]), tensor(-0.6249))

In [427]:
# batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens = model.rollout()

In [428]:
# Test if your agent can at least run episodes
model = PPO(FeedForwardNN, env)
obs, acts, old_logp, rtgs, lens = model.rollout(timesteps_per_batch=200)
print(f"Collected {len(obs)} steps across {len(lens)} episodes")
print(f"Average episode length: {sum(lens)/len(lens):.1f}")
print(f"RTG range: {rtgs.min():.1f} to {rtgs.max():.1f}")

2025-06-04 18:18:38.668319
0
Collected 200 steps across 1 episodes
Average episode length: 200.0
RTG range: -76.7 to -7.0


In [429]:
# MultivariateNormal(model.actor(batch_obs), model.cov_mat).log_prob(batch_acts.unsqueeze(-1))

In [430]:
# model.evaluate(batch_obs, batch_acts)

In [431]:
model.learn()

2025-06-04 18:18:45.505193
0
2025-06-04 18:18:52.444909
200
2025-06-04 18:18:59.552336
400
2025-06-04 18:19:06.378688
600
2025-06-04 18:19:13.201594
800
2025-06-04 18:19:20.029394
1000
Iteration 0: Timesteps 1200, Avg Return: -1041.60, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:19:26.966748
0
2025-06-04 18:19:33.762868
200
2025-06-04 18:19:40.586557
400
2025-06-04 18:19:47.416125
600
2025-06-04 18:19:54.242681
800
2025-06-04 18:20:01.058611
1000
Iteration 1: Timesteps 2400, Avg Return: -1353.75, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:20:07.949266
0
2025-06-04 18:20:14.754717
200
2025-06-04 18:20:21.582350
400
2025-06-04 18:20:28.410878
600
2025-06-04 18:20:35.232119
800
2025-06-04 18:20:42.065319
1000
Iteration 2: Timesteps 3600, Avg Return: -1414.84, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:20:48.984252
0
2025-06-04 18:20:55.775637
200
2

2025-06-04 18:36:41.481238
200
2025-06-04 18:36:48.313761
400
2025-06-04 18:36:55.140249
600
2025-06-04 18:37:01.972091
800
2025-06-04 18:37:08.799956
1000
Iteration 26: Timesteps 32400, Avg Return: -992.61, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:37:15.685442
0
2025-06-04 18:37:22.488072
200
2025-06-04 18:37:29.318229
400
2025-06-04 18:37:36.144952
600
2025-06-04 18:37:43.016364
800
2025-06-04 18:37:49.851075
1000
Iteration 27: Timesteps 33600, Avg Return: -1153.51, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:37:56.743140
0
2025-06-04 18:38:03.537893
200
2025-06-04 18:38:10.355268
400
2025-06-04 18:38:17.188989
600
2025-06-04 18:38:24.023223
800
2025-06-04 18:38:30.848053
1000
Iteration 28: Timesteps 34800, Avg Return: -1166.19, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:38:37.740972
0
2025-06-04 18:38:44.569748
200
2025-06-04 18:38:51.40174

2025-06-04 18:54:46.671414
400
2025-06-04 18:54:53.505381
600
2025-06-04 18:55:00.328778
800
2025-06-04 18:55:07.307378
1000
Iteration 52: Timesteps 63600, Avg Return: -1161.59, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:55:14.499053
0
2025-06-04 18:55:21.294675
200
2025-06-04 18:55:28.248431
400
2025-06-04 18:55:35.070893
600
2025-06-04 18:55:41.900767
800
2025-06-04 18:55:48.743902
1000
Iteration 53: Timesteps 64800, Avg Return: -1116.57, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:55:55.630438
0
2025-06-04 18:56:02.423344
200
2025-06-04 18:56:09.248012
400
2025-06-04 18:56:16.072162
600
2025-06-04 18:56:22.895982
800
2025-06-04 18:56:29.728217
1000
Iteration 54: Timesteps 66000, Avg Return: -1058.43, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 18:56:36.613600
0
2025-06-04 18:56:43.407699
200
2025-06-04 18:56:50.232899
400
2025-06-04 18:56:57.0595

2025-06-04 19:12:44.721232
600
2025-06-04 19:12:51.554212
800
2025-06-04 19:12:58.383931
1000
Iteration 78: Timesteps 94800, Avg Return: -850.71, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 19:13:05.276293
0
2025-06-04 19:13:12.068645
200
2025-06-04 19:13:18.900877
400
2025-06-04 19:13:25.732849
600
2025-06-04 19:13:32.550896
800
2025-06-04 19:13:39.371649
1000
Iteration 79: Timesteps 96000, Avg Return: -874.19, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 19:13:46.271919
0
2025-06-04 19:13:53.078174
200
2025-06-04 19:13:59.901283
400
2025-06-04 19:14:06.722392
600
2025-06-04 19:14:13.560557
800
2025-06-04 19:14:20.378698
1000
Iteration 80: Timesteps 97200, Avg Return: -782.67, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 19:14:27.262068
0
2025-06-04 19:14:34.078943
200
2025-06-04 19:14:41.048944
400
2025-06-04 19:14:48.064416
600
2025-06-04 19:14:55.051568


In [432]:
model.learn()

2025-06-04 20:59:15.047790
0
2025-06-04 20:59:22.639522
200
2025-06-04 20:59:29.456657
400
2025-06-04 20:59:36.281960
600
2025-06-04 20:59:43.106899
800
2025-06-04 20:59:49.935316
1000
Iteration 0: Timesteps 1200, Avg Return: -835.83, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 20:59:56.896980
0
2025-06-04 21:00:03.692933
200
2025-06-04 21:00:10.515933
400
2025-06-04 21:00:17.341687
600
2025-06-04 21:00:24.220705
800
2025-06-04 21:00:31.049234
1000
Iteration 1: Timesteps 2400, Avg Return: -786.57, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:00:37.923508
0
2025-06-04 21:00:44.717175
200
2025-06-04 21:00:51.534810
400
2025-06-04 21:00:58.375531
600
2025-06-04 21:01:05.207072
800
2025-06-04 21:01:14.160936
1000
Iteration 2: Timesteps 3600, Avg Return: -793.55, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:01:22.060116
0
2025-06-04 21:01:28.842577
200
2025

2025-06-04 21:17:28.739263
400
2025-06-04 21:17:35.567045
600
2025-06-04 21:17:42.390070
800
2025-06-04 21:17:49.214184
1000
Iteration 26: Timesteps 32400, Avg Return: -240.39, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:17:56.090809
0
2025-06-04 21:18:02.892099
200
2025-06-04 21:18:09.718153
400
2025-06-04 21:18:16.537986
600
2025-06-04 21:18:23.367790
800
2025-06-04 21:18:30.197363
1000
Iteration 27: Timesteps 33600, Avg Return: -262.25, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:18:37.063332
0
2025-06-04 21:18:43.854763
200
2025-06-04 21:18:50.681863
400
2025-06-04 21:18:57.509046
600
2025-06-04 21:19:04.331987
800
2025-06-04 21:19:11.159918
1000
Iteration 28: Timesteps 34800, Avg Return: -437.04, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:19:18.032131
0
2025-06-04 21:19:25.638355
200
2025-06-04 21:19:32.456628
400
2025-06-04 21:19:40.404006


2025-06-04 21:35:25.140417
600
2025-06-04 21:35:31.955485
800
2025-06-04 21:35:38.780069
1000
Iteration 52: Timesteps 63600, Avg Return: -248.48, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:35:45.671730
0
2025-06-04 21:35:52.471572
200
2025-06-04 21:35:59.300256
400
2025-06-04 21:36:06.125317
600
2025-06-04 21:36:12.950525
800
2025-06-04 21:36:19.780388
1000
Iteration 53: Timesteps 64800, Avg Return: -347.33, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:36:26.674128
0
2025-06-04 21:36:33.470201
200
2025-06-04 21:36:40.358726
400
2025-06-04 21:36:47.339370
600
2025-06-04 21:36:54.254901
800
2025-06-04 21:37:01.083195
1000
Iteration 54: Timesteps 66000, Avg Return: -207.08, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:37:07.974511
0
2025-06-04 21:37:14.771230
200
2025-06-04 21:37:21.600297
400
2025-06-04 21:37:28.446831
600
2025-06-04 21:37:35.262072


2025-06-04 21:53:18.476691
800
2025-06-04 21:53:25.312304
1000
Iteration 78: Timesteps 94800, Avg Return: -130.25, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:53:32.202035
0
2025-06-04 21:53:38.998885
200
2025-06-04 21:53:45.844984
400
2025-06-04 21:53:52.669704
600
2025-06-04 21:53:59.505049
800
2025-06-04 21:54:06.337679
1000
Iteration 79: Timesteps 96000, Avg Return: -243.60, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:54:13.223983
0
2025-06-04 21:54:20.021784
200
2025-06-04 21:54:26.860105
400
2025-06-04 21:54:33.696378
600
2025-06-04 21:54:40.531782
800
2025-06-04 21:54:47.358126
1000
Iteration 80: Timesteps 97200, Avg Return: -244.53, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 21:54:54.244393
0
2025-06-04 21:55:01.050152
200
2025-06-04 21:55:07.896838
400
2025-06-04 21:55:14.720203
600
2025-06-04 21:55:21.562090
800
2025-06-04 21:55:28.387481


In [433]:
model.learn()

2025-06-04 21:59:23.519080
0
2025-06-04 21:59:30.582259
200
2025-06-04 21:59:37.411337
400
2025-06-04 21:59:44.246715
600
2025-06-04 21:59:51.075451
800
2025-06-04 21:59:57.901540
1000
Iteration 0: Timesteps 1200, Avg Return: -228.73, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:00:04.772552
0
2025-06-04 22:00:11.567159
200
2025-06-04 22:00:18.396747
400
2025-06-04 22:00:25.222855
600
2025-06-04 22:00:32.052797
800
2025-06-04 22:00:38.883243
1000
Iteration 1: Timesteps 2400, Avg Return: -175.03, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:00:45.762932
0
2025-06-04 22:00:52.553557
200
2025-06-04 22:00:59.375572
400
2025-06-04 22:01:06.192692
600
2025-06-04 22:01:13.015342
800
2025-06-04 22:01:19.832961
1000
Iteration 2: Timesteps 3600, Avg Return: -171.27, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:01:26.704105
0
2025-06-04 22:01:33.497167
200
2025

2025-06-04 22:17:32.235842
400
2025-06-04 22:17:39.222278
600
2025-06-04 22:17:46.212541
800
2025-06-04 22:17:53.208590
1000
Iteration 26: Timesteps 32400, Avg Return: -232.75, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:18:00.261913
0
2025-06-04 22:18:07.231120
200
2025-06-04 22:18:14.227781
400
2025-06-04 22:18:21.224408
600
2025-06-04 22:18:28.214672
800
2025-06-04 22:18:35.219064
1000
Iteration 27: Timesteps 33600, Avg Return: -193.03, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:18:42.264547
0
2025-06-04 22:18:49.232491
200
2025-06-04 22:18:56.225634
400
2025-06-04 22:19:03.223907
600
2025-06-04 22:19:10.211043
800
2025-06-04 22:19:17.201349
1000
Iteration 28: Timesteps 34800, Avg Return: -283.62, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:19:24.259760
0
2025-06-04 22:19:31.224249
200
2025-06-04 22:19:38.221870
400
2025-06-04 22:19:45.220022


2025-06-04 22:35:50.247175
600
2025-06-04 22:35:57.245363
800
2025-06-04 22:36:04.236448
1000
Iteration 52: Timesteps 63600, Avg Return: -239.89, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:36:11.298255
0
2025-06-04 22:36:18.252737
200
2025-06-04 22:36:25.227328
400
2025-06-04 22:36:32.215051
600
2025-06-04 22:36:39.204329
800
2025-06-04 22:36:46.183871
1000
Iteration 53: Timesteps 64800, Avg Return: -150.88, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:36:53.253274
0
2025-06-04 22:37:00.207644
200
2025-06-04 22:37:07.201043
400
2025-06-04 22:37:14.189749
600
2025-06-04 22:37:21.171123
800
2025-06-04 22:37:28.155601
1000
Iteration 54: Timesteps 66000, Avg Return: -166.93, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:37:35.204276
0
2025-06-04 22:37:42.165952
200
2025-06-04 22:37:49.154750
400
2025-06-04 22:37:56.133997
600
2025-06-04 22:38:03.122754


2025-06-04 22:53:52.335777
800
2025-06-04 22:53:59.160012
1000
Iteration 78: Timesteps 94800, Avg Return: -105.48, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:54:06.025416
0
2025-06-04 22:54:12.817480
200
2025-06-04 22:54:19.637794
400
2025-06-04 22:54:26.456702
600
2025-06-04 22:54:33.284382
800
2025-06-04 22:54:40.111579
1000
Iteration 79: Timesteps 96000, Avg Return: -169.36, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:54:46.987372
0
2025-06-04 22:54:53.776448
200
2025-06-04 22:55:00.602724
400
2025-06-04 22:55:07.418882
600
2025-06-04 22:55:14.240539
800
2025-06-04 22:55:21.053071
1000
Iteration 80: Timesteps 97200, Avg Return: -201.35, Episodes: 6
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
2025-06-04 22:55:27.925145
0
2025-06-04 22:55:34.712475
200
2025-06-04 22:55:41.538432
400
2025-06-04 22:55:48.363539
600
2025-06-04 22:55:55.178937
800
2025-06-04 22:56:02.010930


In [ ]:
# this seems to take a really long time...
# can we not "nudge" the actor to start off with some sort of behaviour?

In [ ]:
# reward

In [116]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

In [117]:
class FeedForwardNN(nn.Module):
    def __init__(self, in_dim, out_dim): # got in_dim hint
        super(FeedForwardNN, self).__init__()
        self.layer1 = nn.Linear(in_dim, 64) # got nn.Linear hint
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, out_dim)
    def forward(self, x):
        x = F.relu(self.layer1(x)) # NOT nn.ReLU - which is the actual layer
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [118]:
# testNN = FeedForwardNN(5,6)

In [119]:
# testNN(torch.tensor([1,2,3,4,5]).to(torch.float32))

Comments:
- This converged very slowly, which is probably normal 
    - Similar to "monkey on a typewriter" where the agent keeps exploring until it hits larger rewards
    - Is this characteristic of on-policy algorithms?
- After a flat line, agent eventually learns and converges (flattens at <-1200, then decreases)
- The reward system here (I'm guessing) is the proximity to the 90° angle integrated over time
- Lastly, this algorithm is an approximation of the RLHF value function (see Williams' REINFORCE and Sutton)
    - But how exactly does the approximation count as PPO, if it appeared first?

Comments (Pt 2):
    - Differences between this algorithm and theoretical RLHF
        - Value function here forgets the regularizing KL divergence term (with respect to reference policy)
        - 


Questions:
- What's the distinction between off-policy and on-policy algorithms?

Next steps:
- Try this on a sparser reward system, play with different reward systems